# Simple API Call


### Setup
Load the API keys and relevant Python libaries. Create an OpenAI client.

In [1]:
import json, dotenv
from openai import OpenAI

openai_api_key = (dotenv.dotenv_values()).get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)


### Create the prompt

Let's assume you are running [AI Driven Telemetry](https://www.cisco.com/c/en/us/td/docs/routers/asr9000/software/asr9k-r7-3/telemetry/configuration/guide/b-telemetry-cg-asr9000-73x/AI-driven-telemetry.html) (ADT). ADT detected a change in the router's state and provided a list of sensor paths which ADT sees as descriptive for the change.

Let's ask GPT-4 to create a diagnosis from the list of sensor paths provided.


In [2]:
sensor_paths = f"""\
Cisco-IOS-XR-ip-bfd-oper:bfd_counters_packet-counters_packet-counter.csv:bfd-mgmt-pkt-display-type-none:HundredGigE0/0/0/16:0/0/CPU0:hello-receive-count
Cisco-IOS-XR-ip-bfd-oper:bfd_session-briefs_session-brief.csv:172.31.14.48:HundredGigE0/0/0/16:0/0/CPU0:0/0/CPU0:ip-single-hop:status-brief-information__async-interval-multiplier__negotiated-local-transmit-interval
Cisco-IOS-XR-ip-bfd-oper:bfd_session-briefs_session-brief.csv:172.31.14.48:HundredGigE0/0/0/16:0/0/CPU0:0/0/CPU0:ip-single-hop:status-brief-information__async-interval-multiplier__negotiated-remote-transmit-interval
Cisco-IOS-XR-ip-bfd-oper:bfd_summary.csv:::session-state__down-count
Cisco-IOS-XR-ip-bfd-oper:bfd_summary.csv:::session-state__up-count
"""

system_prompt = f"""\
You are a networking expert.
You are diagnosing a network issue based on telemetry information received from a Cisco router running IOS-XR.
"""

user_prompt = f"""\
Below, in the "List of sensor paths" section you find a list of YANG sensor path counters which \
have changed the most while the issue occurred. 
The sensor path counters are descriptive of the issue.

Perform the following two steps one after the other:

1. First, explain what is the issue with this router in less than 100 words. Be technical and specific.
   Tag your explanation with the word ISSUE.

2. What do you suggest as next steps to fix the issue?
   Tag your suggestion with the word  RESOLUTION.

Format the output at JSON compliant to RFC 8259. Do not return anything in your response outside of curly braces.

### List of sensor paths
{sensor_paths}
"""

### Call GPT

In [3]:
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='{\n"ISSUE": "Based on the sensor path counters, it appears the router is experiencing Bidirectional Forwarding Detection (BFD) session instability on interface HundredGigE0/0/0/16 with the remote IP 172.31.14.48. The BFD session is frequently transitioning between up and down states, indicated by the changes in \'session-state__down-count\' and \'session-state__up-count\', and changes in the \'hello-receive-count\'. Also, the changing \'negotiated-local-transmit-interval\' and \'negotiated-remote-transmit-interval\' suggest unstable round-trip times.",\n"RESOLUTION": "To resolve this, check if there are network congestion or latency issues affecting BFD packet transmission. Validate the BFD configurations and reconfigure if necessary with higher timers. If issues still persist, isolate if it’s a physical layer problem on interface HundredGigE0/0/0/16 by running interface diagnostics tests and checking for any physical layer errors."\n}', role='assistant',

### Output the result in pretty format

In [4]:
result = json.loads(completion.choices[0].message.content)

In [5]:
for key in result:
    print(key, ": \n", result[key], "\n")

ISSUE : 
 Based on the sensor path counters, it appears the router is experiencing Bidirectional Forwarding Detection (BFD) session instability on interface HundredGigE0/0/0/16 with the remote IP 172.31.14.48. The BFD session is frequently transitioning between up and down states, indicated by the changes in 'session-state__down-count' and 'session-state__up-count', and changes in the 'hello-receive-count'. Also, the changing 'negotiated-local-transmit-interval' and 'negotiated-remote-transmit-interval' suggest unstable round-trip times. 

RESOLUTION : 
 To resolve this, check if there are network congestion or latency issues affecting BFD packet transmission. Validate the BFD configurations and reconfigure if necessary with higher timers. If issues still persist, isolate if it’s a physical layer problem on interface HundredGigE0/0/0/16 by running interface diagnostics tests and checking for any physical layer errors. 

